## 1.3 Processing SignalP Output

### Overview
After generating initial predictions with SignalP, the output need to be processed to extract and format useful data for further analysis. This step involves parsing the SignalP output file to retrieve detailed information about each predicted signal peptide.

### Install requirement
 


In [ ]:
# Install Biopython, which provides tools for biological computation
!pip install biopython

# Criteria and funtions

Source: [ProtParam module](https://biopython.org/docs/1.75/api/Bio.SeqUtils.ProtParam.html)

 Methods used: **gravy(self)** --> Calculate the gravy according to Kyte and Doolittle.


In [ ]:
def regions(dire):
    """
    This function extracts sequence regions from a specified results directory.
    
    Args:
    dire (str): Sub-directory under 'Results_toxin/' containing the result files.
    
    Returns:
    list: Contains regions 'N', 'H', 'C', the full signal peptide (SP) sequence, 
          the first 16 characters of the 'Pro' region, and the complete SP with 'Pro' region.
    """

    #Path to the folder where results are stored
    # Change the path to the folder
    link = 'Results_toxin/' + dire

    import csv
    matrix = []
    cont = 0
    # Reading the tab-delimited file
    with open(link, 'r') as file:
        reader = csv.reader(file, delimiter = '\t')
        for row in reader:
            matrix.append(row)
        # Ignore the first two header lines
        matrix = matrix[2::]

    matrix = [list(i) for i in zip(*matrix)]
    # Initialize region strings
    n_region = ''
    h_region = ''
    c_region = ''
    pro_region = ''

    cont = 0
    # Extract regions based on column values
    for i in range(len(matrix[0])):

        if matrix[2][i] == 'N':
            n_region = n_region + matrix[1][i]

        elif matrix[2][i] == 'H':
            h_region = h_region + matrix[1][i]

        elif matrix[2][i] == 'C':
            c_region = c_region + matrix[1][i]

        elif matrix[2][i] == 'O':
            pro_region = pro_region + matrix[1][i]

    # Concatenate N, H, C regions to form the full signal peptide
    full_sp = n_region + h_region + c_region
    # Concatenate full SP with Pro region
    complete_sp_pro = full_sp + pro_region

    return [n_region, h_region, c_region, full_sp, pro_region[0:16], complete_sp_pro]



def gravy(seq): 
    """
    Calculate the hydrophobicity index of a sequence using the Kyte & Doolittle method.
    
    Args:
    seq (str): Amino acid sequence for which to calculate the hydrophobicity.
    
    Returns:
    float: Hydrophobicity index rounded to two decimal places.
    """
    # Hydrophobicity scale from Kyte & Doolittle. J. Mol. Biol. 157:105-132(1982).
    g_kd = {
        "A": 1.8, "R": -4.5, "N": -3.5, "D": -3.5, "C": 2.5,
        "Q": -3.5, "E": -3.5, "G": -0.4, "H": -3.2, "I": 4.5,
        "L": 3.8, "K": -3.9, "M": 1.9, "F": 2.8, "P": -1.6,
        "S": -0.8, "T": -0.7, "W": -0.9, "Y": -1.3, "V": 4.2
    }

    kd = 0
    for i in seq:
        for res, resGrav in g_kd.items():
            if i == res:
                kd = kd + resGrav
    kd=kd/len(seq)
    return round(kd,2)


def name_select(name):
    """
    Extract the unique identifier from a given UniProt entry name.
    
    Args:
    name (str): Full UniProt entry name including pipes and other metadata.
    
    Returns:
    str: Extracted unique identifier (ID) from the name.
    """
    # Find positions of '|' characters to delineate the ID
    cod = []
    for n,l in enumerate(name):
        if name[n] == '|':
            cod.append(n)
    # Extract and return the ID part
    ID = name[cod[0]+1:cod[1]]

    return ID



def charge(seq):
    """
    Calculate the net charge of a sequence based on the presence of specific amino acids
    
    Args:
    seq (str): Amino acid sequence.
    """
    suma = 0

    for i in seq:
        if i in ['K', 'R']:
            suma = suma + 1

        if i == 'H':
            suma = suma + 0.1
        elif i in ['D', 'E']:
            suma = suma -1

    return suma


### Parsing Output

This part processes the output file "prediction_results_toxin.txt" from SignalP to extract essential information such as peptide scores, cleavage site positions, and prediction confidence.

In [ ]:
import os
import csv

# Initialize an empty list to hold all rows from the CSV file
matrix = []
cont = 0

# Open the prediction result file from SignalP
with open('Results_toxin/prediction_results_toxin.txt', 'r') as file:
    reader = csv.reader(file, delimiter = '\t')
    for row in reader:
        matrix.append(row) # Append each row of data to the matrix list
        # Optionally print each row, currently commented out
        #print(row)

# Remove the first two lines
matrix = matrix[2::]
# Display the resulting matrix
matrix

[['sp|A0A0N9NCU6|YOPJ_YERPU_Serine/threonine-protein_acetyltransferase_YopJ_OS=Yer',
  'NO_SP',
  '1.000029',
  '0.000027',
  '0.000000',
  '0.000000',
  '0.000000',
  '0.000000',
  ''],
 ['sp|A0A1S4NYE3|CDIA_ECOST_tRNA_nuclease_CdiA_OS=Escherichia_coli_(strain_STEC_O3',
  'SP',
  '0.024348',
  '0.974034',
  '0.000681',
  '0.000438',
  '0.000238',
  '0.000230',
  'CS pos: 32-33. Pr: 0.8066'],
 ['sp|A0A2S3R7M0|MARTX_VIBVL_Multifunctional-autoprocessing_repeats-in-toxin_OS=Vi',
  'NO_SP',
  '1.000025',
  '0.000005',
  '0.000000',
  '0.000000',
  '0.000000',
  '0.000000',
  ''],
 ['sp|A0A482PDI9|NLEB_CITRO_Protein-arginine_N-acetylglucosaminyltransferase_NleB_',
  'NO_SP',
  '1.000047',
  '0.000000',
  '0.000000',
  '0.000000',
  '0.000000',
  '0.000000',
  ''],
 ['sp|A1YKW7|RTXA_KINKI_Cytolysin_RtxA_OS=Kingella_kingae_OX=504_GN=rtxA_PE=1_SV=1',
  'NO_SP',
  '1.000077',
  '0.000000',
  '0.000000',
  '0.000000',
  '0.000000',
  '0.000000',
  ''],
 ['sp|B3BM80|CDIA4_ECO5C_Deoxyribonuclease_

In [ ]:
# To recognize name
def name_select(name):
    # Initialize an empty list
    cod = []
    for n,l in enumerate(name):
        # If the character is '|', append the index to 'cod'
        if name[n] == '|':
            cod.append(n)

    ID = name[cod[0]+1:cod[1]]
    # Return the extracted ID
    return ID

In [ ]:
# The name and SP percentage column is selected
import os
# List to store file names within the directory
dir_archivos = []

with os.scandir('Results_toxin') as ficheros:
    for fichero in ficheros:
        dir_archivos.append(fichero.name)
        # Optionally print each file name
        #print(fichero.name)

# List to store data related to signal peptides
data_sp = []

# Iterate over each row in the matrix processed earlier
i = 0
for fila in matrix:
    i= i+ 1
    # Optionally print 
    #print(fila)
  
  
    # Filter rows where the second column indicates a signal peptide ("SP")
    if str(fila[1]) == "SP":
        name = '_'+name_select(fila[0])+ '_' # the file name is extracted
        name_dir = [s for s in dir_archivos if name in s][0]

        # file is accessed if name is found in path
        pep_regions = regions(name_dir)


        if "X" not in pep_regions[3] and "" not in pep_regions:

            region_n = pep_regions[0]    # N-region
            region_h = pep_regions[1]    # H-region
            #print("EEEEEEEEEEE", name, pep_regions)
            region_c = pep_regions[2]    # C-region
            SP       = pep_regions[3]    # Full Signal Peptide
            len_sp = len(pep_regions[3]) # Length of the signal peptide
            region_pro = pep_regions[4]  # Pro region
            seq_70aa = pep_regions[5]    # Complete fragment (70 amino acids)

            # Calculate net charge of the N-region

            carga_nlit = charge(region_n)

            # Calculate hydrophobicity of the H-region and full signal peptide
            hidro_h = ProteinAnalysis(region_h).gravy()
            hidro_sp = ProteinAnalysis(SP).gravy()


            # Compile all extracted and computed data into a list
            sp_fila = [name_select(fila[0]),     # Uniprot ID

                          fila[1],    # ID  (SP-> signal peptide)
                          fila[3],    # SP-likelihood 
                          region_n  , # N-region
                          region_h  , # H-region
                          region_c  , # C-region
                          SP,         # Full Signal Peptide
                          len_sp,     # Length of the signal peptide
                          region_pro, # Pro-region
                          seq_70aa,   # Complete fragment (70 amino acids)
                          #carga_n,
                          carga_nlit, # Net charge of N-region
                       hidro_h,       # Hydrophobicity of H-region
                       hidro_sp       # Hydrophobicity of full signal peptide
                      ]
        else:
            sp_fila = np.empty((1,13))
        
        print(sp_fila)
        # Append the row to the main data list
        data_sp.append(sp_fila)


EEEEEEEEEEE _A0A1S4NYE3_ ['MHQPPVRFPYR', 'LLSYLISTIIAGQPLL', 'PAVGA', 'MHQPPVRFPYRLLSYLISTIIAGQPLLPAVGA', 'VITPQNGAGMDKAANG', 'MHQPPVRFPYRLLSYLISTIIAGQPLLPAVGAVITPQNGAGMDKAANGVPVVNIATPNGAGISHNRFTDY']
['A0A1S4NYE3', 'SP', '0.974034', 'MHQPPVRFPYR', 'LLSYLISTIIAGQPLL', 'PAVGA', 'MHQPPVRFPYRLLSYLISTIIAGQPLLPAVGA', 32, 'VITPQNGAGMDKAANG', 'MHQPPVRFPYRLLSYLISTIIAGQPLLPAVGAVITPQNGAGMDKAANGVPVVNIATPNGAGISHNRFTDY', 2.1, 1.5750000000000002, 0.5656250000000002]
EEEEEEEEEEE _D5CBA0_ ['MMKQDQVRFSQR', 'ALSALLSVLLATQPLL', 'PAVA', 'MMKQDQVRFSQRALSALLSVLLATQPLLPAVA', 'ASITPSGNTQMDKAAN', 'MMKQDQVRFSQRALSALLSVLLATQPLLPAVAASITPSGNTQMDKAANGVPVVNIATPNQSGISHNKYND']
['D5CBA0', 'SP', '0.998014', 'MMKQDQVRFSQR', 'ALSALLSVLLATQPLL', 'PAVA', 'MMKQDQVRFSQRALSALLSVLLATQPLLPAVA', 32, 'ASITPSGNTQMDKAAN', 'MMKQDQVRFSQRALSALLSVLLATQPLLPAVAASITPSGNTQMDKAANGVPVVNIATPNQSGISHNKYND', 2, 1.8, 0.5656250000000003]
EEEEEEEEEEE _P01555_ ['MVK', 'IIFVFFIFLSSF', 'SYA', 'MVKIIFVFFIFLSSFSYA', 'NDDKLYRADSRPPDEI', 'MVKIIFVFFIFLSSFSYA

EEEEEEEEEEE _Q48258_ ['MEIQQTHRKINR', 'PLVSLALVGALVSIT', 'PQQSHA', 'MEIQQTHRKINRPLVSLALVGALVSITPQQSHA', 'AFFTTVIIPAIVGGIA', 'MEIQQTHRKINRPLVSLALVGALVSITPQQSHAAFFTTVIIPAIVGGIATGAAVGTVSGLLGWGLKQAEE']
['Q48258', 'SP', '0.935472', 'MEIQQTHRKINR', 'PLVSLALVGALVSIT', 'PQQSHA', 'MEIQQTHRKINRPLVSLALVGALVSITPQQSHA', 33, 'AFFTTVIIPAIVGGIA', 'MEIQQTHRKINRPLVSLALVGALVSITPQQSHAAFFTTVIIPAIVGGIATGAAVGTVSGLLGWGLKQAEE', 2.1, 2.1066666666666665, 0.027272727272727372]
EEEEEEEEEEE _Q57478_ ['MKC', 'ILLKWILCLLLGFSSV', 'SYS', 'MKCILLKWILCLLLGFSSVSYS', 'QEFTIDFSTQQSYVSS', 'MKCILLKWILCLLLGFSSVSYSQEFTIDFSTQQSYVSSLNSIRTAISTPLEHISQGATSVSVINHTPPGS']
['Q57478', 'SP', '0.999182', 'MKC', 'ILLKWILCLLLGFSSV', 'SYS', 'MKCILLKWILCLLLGFSSVSYS', 22, 'QEFTIDFSTQQSYVSS', 'MKCILLKWILCLLLGFSSVSYSQEFTIDFSTQQSYVSSLNSIRTAISTPLEHISQGATSVSVINHTPPGS', 1, 2.1562500000000004, 1.4590909090909099]
EEEEEEEEEEE _Q8KTU8_ ['MKC', 'ILFKWVLCLLLGFSSV', 'SYS', 'MKCILFKWVLCLLLGFSSVSYS', 'REFTIDFSTQQSYVSS', 'MKCILFKWVLCLLLGFSSVSYSREFTIDFSTQQSYVS

EEEEEEEEEEE _A0A0F4NLB3_ ['MNSHLILLNVRKNNMNNNK', 'VVNITLVSAAVSLGLF', 'STNAFA', 'MNSHLILLNVRKNNMNNNKVVNITLVSAAVSLGLFSTNAFA', 'AVDILDIKLQGQSCEM', 'MNSHLILLNVRKNNMNNNKVVNITLVSAAVSLGLFSTNAFAAVDILDIKLQGQSCEMGFSPIAADEVMPI']
['A0A0F4NLB3', 'SP', '0.998838', 'MNSHLILLNVRKNNMNNNK', 'VVNITLVSAAVSLGLF', 'STNAFA', 'MNSHLILLNVRKNNMNNNKVVNITLVSAAVSLGLFSTNAFA', 41, 'AVDILDIKLQGQSCEM', 'MNSHLILLNVRKNNMNNNKVVNITLVSAAVSLGLFSTNAFAAVDILDIKLQGQSCEMGFSPIAADEVMPI', 3.1, 2.05625, 0.42439024390243907]
EEEEEEEEEEE _A0A0F6WGF3_ ['MHQPPVRFPYR', 'LLSYLISTIIAGQPLL', 'PAVGA', 'MHQPPVRFPYRLLSYLISTIIAGQPLLPAVGA', 'VITPQNGAGMDKAANG', 'MHQPPVRFPYRLLSYLISTIIAGQPLLPAVGAVITPQNGAGMDKAANGVPVVNIATPNGAGISHNRFTDY']
['A0A0F6WGF3', 'SP', '0.974034', 'MHQPPVRFPYR', 'LLSYLISTIIAGQPLL', 'PAVGA', 'MHQPPVRFPYRLLSYLISTIIAGQPLLPAVGA', 32, 'VITPQNGAGMDKAANG', 'MHQPPVRFPYRLLSYLISTIIAGQPLLPAVGAVITPQNGAGMDKAANGVPVVNIATPNGAGISHNRFTDY', 2.1, 1.5750000000000002, 0.5656250000000002]
EEEEEEEEEEE _A0A0G4K079_ ['MKPIKTTQR', 'LMAYTLINLIAFQPVL', 'PA

EEEEEEEEEEE _A0A0T9QJY3_ ['MKQNKFKLSPAGK', 'LAAAVAIISVSVAT', 'CYA', 'MKQNKFKLSPAGKLAAAVAIISVSVATCYA', 'AGIVGAGDSAHRPEVN', 'MKQNKFKLSPAGKLAAAVAIISVSVATCYAAGIVGAGDSAHRPEVNSVNGVSVVDIVKPSASGLSHNQFD']
['A0A0T9QJY3', 'SP', '0.997368', 'MKQNKFKLSPAGK', 'LAAAVAIISVSVAT', 'CYA', 'MKQNKFKLSPAGKLAAAVAIISVSVATCYA', 30, 'AGIVGAGDSAHRPEVN', 'MKQNKFKLSPAGKLAAAVAIISVSVATCYAAGIVGAGDSAHRPEVNSVNGVSVVDIVKPSASGLSHNQFD', 4, 2.2928571428571423, 0.6666666666666665]
EEEEEEEEEEE _A0A0T9R5B6_ ['MKTHTFKLSPAGK', 'LAAAVTIISVSVAT', 'CYA', 'MKTHTFKLSPAGKLAAAVTIISVSVATCYA', 'AGIVGAGDSAHKPDVS', 'MKTHTFKLSPAGKLAAAVTIISVSVATCYAAGIVGAGDSAHKPDVSSVNGTSVINIVEPSASGLSHNKFQ']
['A0A0T9R5B6', 'SP', '0.996090', 'MKTHTFKLSPAGK', 'LAAAVTIISVSVAT', 'CYA', 'MKTHTFKLSPAGKLAAAVTIISVSVATCYA', 30, 'AGIVGAGDSAHKPDVS', 'MKTHTFKLSPAGKLAAAVTIISVSVATCYAAGIVGAGDSAHKPDVSSVNGTSVINIVEPSASGLSHNKFQ', 3.1, 2.1142857142857143, 0.7933333333333332]
EEEEEEEEEEE _A0A0T9SRS8_ ['MIPIYFRQK', 'LISYALIYLVAIQPIM', 'PVMA', 'MIPIYFRQKLISYALIYLVAIQPIMPVMA', 'AGIDV

EEEEEEEEEEE _A0A1I4XC96_ ['MDDQLTLSLPRR', 'LLSYLICSLIAFQPLL', 'PAFS', 'MDDQLTLSLPRRLLSYLICSLIAFQPLLPAFS', 'AAIAPVTPGTKVDAAG', 'MDDQLTLSLPRRLLSYLICSLIAFQPLLPAFSAAIAPVTPGTKVDAAGNGVPVINIATPNAAGLSHNQYQ']
['A0A1I4XC96', 'SP', '0.946707', 'MDDQLTLSLPRR', 'LLSYLICSLIAFQPLL', 'PAFS', 'MDDQLTLSLPRRLLSYLICSLIAFQPLLPAFS', 32, 'AAIAPVTPGTKVDAAG', 'MDDQLTLSLPRRLLSYLICSLIAFQPLLPAFSAAIAPVTPGTKVDAAGNGVPVINIATPNAAGLSHNQYQ', 0, 1.9312500000000001, 0.74375]
EEEEEEEEEEE _A0A1I6E059_ ['MKTIKKDKITESNLHLR', 'LAPLYLSLASIFSPL', 'AEA', 'MKTIKKDKITESNLHLRLAPLYLSLASIFSPLAEA', 'AGTVPDPRQPGGPTMG', 'MKTIKKDKITESNLHLRLAPLYLSLASIFSPLAEAAGTVPDPRQPGGPTMGKTANDTPMVNIVNPNAKGV']
['A0A1I6E059', 'SP', '0.998860', 'MKTIKKDKITESNLHLR', 'LAPLYLSLASIFSPL', 'AEA', 'MKTIKKDKITESNLHLRLAPLYLSLASIFSPLAEA', 35, 'AGTVPDPRQPGGPTMG', 'MKTIKKDKITESNLHLRLAPLYLSLASIFSPLAEAAGTVPDPRQPGGPTMGKTANDTPMVNIVNPNAKGV', 3.1, 1.533333333333333, 0.1600000000000001]
EEEEEEEEEEE _A0A1K0JBT9_ ['MNQNRYRLVFNKQRGMLMAVPECANGAHKAASGERAGDTGRSFLAT', 'LRPIAWALLLSTG

...

EEEEEEEEEEE _A0A6M8M6V7_ ['MDVRQFAFLARQPSAALKPRDSFFGLPKRG', 'LVLILINALFWQPL', 'LAQA', 'MDVRQFAFLARQPSAALKPRDSFFGLPKRGLVLILINALFWQPLLAQA', 'EGIVVSAPGTTVGQAG', 'MDVRQFAFLARQPSAALKPRDSFFGLPKRGLVLILINALFWQPLLAQAEGIVVSAPGTTVGQAGNGVPVV']
['A0A6M8M6V7', 'SP', '0.818962', 'MDVRQFAFLARQPSAALKPRDSFFGLPKRG', 'LVLILINALFWQPL', 'LAQA', 'MDVRQFAFLARQPSAALKPRDSFFGLPKRGLVLILINALFWQPLLAQA', 48, 'EGIVVSAPGTTVGQAG', 'MDVRQFAFLARQPSAALKPRDSFFGLPKRGLVLILINALFWQPLLAQAEGIVVSAPGTTVGQAGNGVPVV', 4, 1.9500000000000004, 0.41875]
EEEEEEEEEEE _A0A6M8SYA4_ ['MKKN', 'IWMIVISTGLALGGA', 'PTEGIVA', 'MKKNIWMIVISTGLALGGAPTEGIVA', 'GTLSPYINTEIKKATE', 'MKKNIWMIVISTGLALGGAPTEGIVAGTLSPYINTEIKKATEGNDTANILAHGVWGAMEAASQGGSALGG']
['A0A6M8SYA4', 'SP', '0.998861', 'MKKN', 'IWMIVISTGLALGGA', 'PTEGIVA', 'MKKNIWMIVISTGLALGGAPTEGIVA', 26, 'GTLSPYINTEIKKATE', 'MKKNIWMIVISTGLALGGAPTEGIVAGTLSPYINTEIKKATEGNDTANILAHGVWGAMEAASQGGSALGG', 2, 1.8133333333333337, 0.8499999999999998]
EEEEEEEEEEE _A0A6M8T0G4_ ['MKK', 'NIWMIVISTGL', 'ALG', 'MKKNIWM

EEEEEEEEEEE _A0A7U9KKI1_ ['MKNNNFRLSGAGK', 'LAASLAIILASLG', 'STYA', 'MKNNNFRLSGAGKLAASLAIILASLGSTYA', 'GEIVAANGANGPGVST', 'MKNNNFRLSGAGKLAASLAIILASLGSTYAGEIVAANGANGPGVSTAGNGAQVVNIVTPNDQGLSHNQYQ']
['A0A7U9KKI1', 'SP', '0.998900', 'MKNNNFRLSGAGK', 'LAASLAIILASLG', 'STYA', 'MKNNNFRLSGAGKLAASLAIILASLGSTYA', 30, 'GEIVAANGANGPGVST', 'MKNNNFRLSGAGKLAASLAIILASLGSTYAGEIVAANGANGPGVSTAGNGAQVVNIVTPNDQGLSHNQYQ', 3, 2.2615384615384615, 0.4766666666666666]
EEEEEEEEEEE _A0A7U9PVJ6_ ['MKHNNNQNLNIPQR', 'LLSYTLCALLAGQPLL', 'PALA', 'MKHNNNQNLNIPQRLLSYTLCALLAGQPLLPALA', 'EGVNVAEGNTRVDQAA', 'MKHNNNQNLNIPQRLLSYTLCALLAGQPLLPALAEGVNVAEGNTRVDQAANGVPVINIATPNQAGISHNK']
['A0A7U9PVJ6', 'SP', '0.998867', 'MKHNNNQNLNIPQR', 'LLSYTLCALLAGQPLL', 'PALA', 'MKHNNNQNLNIPQRLLSYTLCALLAGQPLLPALA', 34, 'EGVNVAEGNTRVDQAA', 'MKHNNNQNLNIPQRLLSYTLCALLAGQPLLPALAEGVNVAEGNTRVDQAANGVPVINIATPNQAGISHNK', 2.1, 1.5250000000000001, 0.0794117647058824]
EEEEEEEEEEE _A0A7V7JCM6_ ['MMKQDQVRFSQR', 'ALSALLSVLLATQPLL', 'PAVA', 'MMKQDQVRFSQRALSALLS

EEEEEEEEEEE _A0A7Z8ZTD8_ ['MVVADIARSGRAGTSLSSRTGYPHRQRICR', 'VTPLAFSLWLASGMV', 'HSVSA', 'MVVADIARSGRAGTSLSSRTGYPHRQRICRVTPLAFSLWLASGMVHSVSA', 'AGIVADHGAPGHQQPT', 'MVVADIARSGRAGTSLSSRTGYPHRQRICRVTPLAFSLWLASGMVHSVSAAGIVADHGAPGHQQPTITQT']
['A0A7Z8ZTD8', 'SP', '0.577654', 'MVVADIARSGRAGTSLSSRTGYPHRQRICR', 'VTPLAFSLWLASGMV', 'HSVSA', 'MVVADIARSGRAGTSLSSRTGYPHRQRICRVTPLAFSLWLASGMVHSVSA', 50, 'AGIVADHGAPGHQQPT', 'MVVADIARSGRAGTSLSSRTGYPHRQRICRVTPLAFSLWLASGMVHSVSAAGIVADHGAPGHQQPTITQT', 5.1, 1.5266666666666664, 0.184]
EEEEEEEEEEE _A0A7Z9D171_ ['MVVADIARSGRAGTSLSSRTGYPHRQRICR', 'VTPLAFSLWLASGMV', 'HSVSA', 'MVVADIARSGRAGTSLSSRTGYPHRQRICRVTPLAFSLWLASGMVHSVSA', 'AGIVADHGAPGHQQPT', 'MVVADIARSGRAGTSLSSRTGYPHRQRICRVTPLAFSLWLASGMVHSVSAAGIVADHGAPGHQQPTITQT']
['A0A7Z9D171', 'SP', '0.577654', 'MVVADIARSGRAGTSLSSRTGYPHRQRICR', 'VTPLAFSLWLASGMV', 'HSVSA', 'MVVADIARSGRAGTSLSSRTGYPHRQRICRVTPLAFSLWLASGMVHSVSA', 50, 'AGIVADHGAPGHQQPT', 'MVVADIARSGRAGTSLSSRTGYPHRQRICRVTPLAFSLWLASGMVHSVSAAGIVADHGAPGHQQPTITQT', 5.

EEEEEEEEEEE _E3PPC0_ ['MKK', 'SILFIFLSVLSF', 'SPFA', 'MKKSILFIFLSVLSFSPFA', 'QDAKPAGSSKEKITLE', 'MKKSILFIFLSVLSFSPFAQDAKPAGSSKEKITLESKKCNIVKKNNESSPESMNSSNYCCELCCNPACTG']
['E3PPC0', 'SP', '0.999169', 'MKK', 'SILFIFLSVLSF', 'SPFA', 'MKKSILFIFLSVLSFSPFA', 19, 'QDAKPAGSSKEKITLE', 'MKKSILFIFLSVLSFSPFAQDAKPAGSSKEKITLESKKCNIVKKNNESSPESMNSSNYCCELCCNPACTG', 2, 2.5500000000000003, 1.4157894736842105]
EEEEEEEEEEE _E6YVQ4_ ['MFR', 'LFITLIIVIFSHF', 'VFA', 'MFRLFITLIIVIFSHFVFA', 'DDDRIVYRAAMDTPKD', 'MFRLFITLIIVIFSHFVFADDDRIVYRAAMDTPKDLKAAGGFFPRGMDRTRPNQPPPDISLWNHVNGTGT']
['E6YVQ4', 'SP', '0.999131', 'MFR', 'LFITLIIVIFSHF', 'VFA', 'MFRLFITLIIVIFSHFVFA', 19, 'DDDRIVYRAAMDTPKD', 'MFRLFITLIIVIFSHFVFADDDRIVYRAAMDTPKDLKAAGGFFPRGMDRTRPNQPPPDISLWNHVNGTGT', 1, 2.576923076923077, 2.2368421052631575]
EEEEEEEEEEE _E9CK44_ ['MGR', 'LAYSVALLFLI', 'RSSACFA', 'MGRLAYSVALLFLIRSSACFA', 'VLPLEVYRSVMEDPEL', 'MGRLAYSVALLFLIRSSACFAVLPLEVYRSVMEDPELVKADDGFLPKGMDGTRPNQPIPSVSLYNHAMGS']
['E9CK44', 'SP', '0.999149', 'MGR', 'LA

EEEEEEEEEEE _V5SKL9_ ['MVK', 'IIFVFFIFLSSF', 'SYA', 'MVKIIFVFFIFLSSFSYA', 'NDDKLYRADSRPPDEI', 'MVKIIFVFFIFLSSFSYANDDKLYRADSRPPDEIKQSGGLMPRGQSEYFDRGTQMNINLYDHARGTQTGF']
['V5SKL9', 'SP', '0.998963', 'MVK', 'IIFVFFIFLSSF', 'SYA', 'MVKIIFVFFIFLSSFSYA', 18, 'NDDKLYRADSRPPDEI', 'MVKIIFVFFIFLSSFSYANDDKLYRADSRPPDEIKQSGGLMPRGQSEYFDRGTQMNINLYDHARGTQTGF', 1, 2.8249999999999997, 1.9888888888888892]
EEEEEEEEEEE _V5YUX5_ ['MQDPTAIYGETWMKNNNFRLSAAGK', 'LAAALAIILAA', 'SAGAYA', 'MQDPTAIYGETWMKNNNFRLSAAGKLAAALAIILAASAGAYA', 'AEIVAANGANGPGVST', 'MQDPTAIYGETWMKNNNFRLSAAGKLAAALAIILAASAGAYAAEIVAANGANGPGVSTAATGAQVVDIVA']
['V5YUX5', 'SP', '0.992186', 'MQDPTAIYGETWMKNNNFRLSAAGK', 'LAAALAIILAA', 'SAGAYA', 'MQDPTAIYGETWMKNNNFRLSAAGKLAAALAIILAASAGAYA', 42, 'AEIVAANGANGPGVST', 'MQDPTAIYGETWMKNNNFRLSAAGKLAAALAIILAASAGAYAAEIVAANGANGPGVSTAATGAQVVDIVA', 1, 2.8363636363636364, 0.3404761904761906]
EEEEEEEEEEE _V8CLS4_ ['MR', 'LAVVVLVLLGV', 'CWA', 'MRLAVVVLVLLGVCWA', 'QEVGVSVDLGIGDSPT', 'MRLAVVVLVLLGVCWAQEVGVSVDLGIGDSPTP

## Output File Generation

The final processed data (data_sp) is saved in a data frame and then in a  xlsx file (Excel file) providing a clean and accessible format. 

In [ ]:
# Import the pandas library
import pandas as pd
# Convert the list of data into a pandas DataFrame
df = pd.DataFrame(data_sp)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,A0A1S4NYE3,SP,0.974034,MHQPPVRFPYR,LLSYLISTIIAGQPLL,PAVGA,MHQPPVRFPYRLLSYLISTIIAGQPLLPAVGA,32.0,VITPQNGAGMDKAANG,MHQPPVRFPYRLLSYLISTIIAGQPLLPAVGAVITPQNGAGMDKAA...,2.1,1.575000,0.565625
1,D5CBA0,SP,0.998014,MMKQDQVRFSQR,ALSALLSVLLATQPLL,PAVA,MMKQDQVRFSQRALSALLSVLLATQPLLPAVA,32.0,ASITPSGNTQMDKAAN,MMKQDQVRFSQRALSALLSVLLATQPLLPAVAASITPSGNTQMDKA...,2.0,1.800000,0.565625
2,P01555,SP,0.998963,MVK,IIFVFFIFLSSF,SYA,MVKIIFVFFIFLSSFSYA,18.0,NDDKLYRADSRPPDEI,MVKIIFVFFIFLSSFSYANDDKLYRADSRPPDEIKQSGGLMPRGQS...,1.0,2.825000,1.988889
3,P01556,SP,0.998953,MIK,LKFGVFFTVLLSS,AYAHG,MIKLKFGVFFTVLLSSAYAHG,21.0,TPQNITDLCAEYHNTQ,MIKLKFGVFFTVLLSSAYAHGTPQNITDLCAEYHNTQIYTLNDKIF...,1.0,1.661538,1.085714
4,P04977,SP,0.998410,MRCTRAIRQTART,GWLTWLAILAVTAPV,TSPAWA,MRCTRAIRQTARTGWLTWLAILAVTAPVTSPAWA,34.0,DDPPATVYRYDSRPPE,MRCTRAIRQTARTGWLTWLAILAVTAPVTSPAWADDPPATVYRYDS...,4.0,1.633333,0.382353
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2197,W6HYF7,SP,0.998774,MTN,FIQTVFKIGLFSIFTFFSQL,SSA,MTNFIQTVFKIGLFSIFTFFSQLSSA,26.0,DIDAELIDDMVQVNPL,MTNFIQTVFKIGLFSIFTFFSQLSSADIDAELIDDMVQVNPLISLR...,0.0,1.390000,0.988462
2198,W6HZS4,SP,0.998695,MTN,FIQTVFKIGLFSIFTFF,SQFSSA,MTNFIQTVFKIGLFSIFTFFSQFSSA,26.0,DIDAELIDDMVQVNPL,MTNFIQTVFKIGLFSIFTFFSQFSSADIDAELIDDMVQVNPLISLR...,0.0,1.664706,0.950000
2199,W6HZU5,SP,0.998769,MRH,YIIYGFLLIACFY,STQA,MRHYIIYGFLLIACFYSTQA,20.0,NSDSLNPNLLTDTPTP,MRHYIIYGFLLIACFYSTQANSDSLNPNLLTDTPTPPGKVGEIVGE...,1.1,2.053846,0.885000
2200,W6IBI6,SP,0.998736,MRH,YIIYGFLLIACFY,STQA,MRHYIIYGFLLIACFYSTQA,20.0,NSDSLNPNLLTDTPTP,MRHYIIYGFLLIACFYSTQANSDSLNPNLLTDTPTPPGKVGEIVGE...,1.1,2.053846,0.885000


In [ ]:
# Remove duplicate rows based on column 6 and drop any rows with missing values

toxin_database = df.drop_duplicates(subset=6).dropna()
toxin_database


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,A0A1S4NYE3,SP,0.974034,MHQPPVRFPYR,LLSYLISTIIAGQPLL,PAVGA,MHQPPVRFPYRLLSYLISTIIAGQPLLPAVGA,32.0,VITPQNGAGMDKAANG,MHQPPVRFPYRLLSYLISTIIAGQPLLPAVGAVITPQNGAGMDKAA...,2.1,1.575000,0.565625
1,D5CBA0,SP,0.998014,MMKQDQVRFSQR,ALSALLSVLLATQPLL,PAVA,MMKQDQVRFSQRALSALLSVLLATQPLLPAVA,32.0,ASITPSGNTQMDKAAN,MMKQDQVRFSQRALSALLSVLLATQPLLPAVAASITPSGNTQMDKA...,2.0,1.800000,0.565625
2,P01555,SP,0.998963,MVK,IIFVFFIFLSSF,SYA,MVKIIFVFFIFLSSFSYA,18.0,NDDKLYRADSRPPDEI,MVKIIFVFFIFLSSFSYANDDKLYRADSRPPDEIKQSGGLMPRGQS...,1.0,2.825000,1.988889
3,P01556,SP,0.998953,MIK,LKFGVFFTVLLSS,AYAHG,MIKLKFGVFFTVLLSSAYAHG,21.0,TPQNITDLCAEYHNTQ,MIKLKFGVFFTVLLSSAYAHGTPQNITDLCAEYHNTQIYTLNDKIF...,1.0,1.661538,1.085714
4,P04977,SP,0.998410,MRCTRAIRQTART,GWLTWLAILAVTAPV,TSPAWA,MRCTRAIRQTARTGWLTWLAILAVTAPVTSPAWA,34.0,DDPPATVYRYDSRPPE,MRCTRAIRQTARTGWLTWLAILAVTAPVTSPAWADDPPATVYRYDS...,4.0,1.633333,0.382353
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2192,W3V795,SP,0.998635,MSKLNNSVVR,GTSYLLIYLTAIQPL,HPAIA,MSKLNNSVVRGTSYLLIYLTAIQPLHPAIA,30.0,AGITPDNNHTQVQNQG,MSKLNNSVVRGTSYLLIYLTAIQPLHPAIAAGITPDNNHTQVQNQG...,2.0,1.046667,0.536667
2193,W3V835,SP,0.999009,MDRCNNPMAR,GACYLLIYLTATYPL,HPAIA,MDRCNNPMARGACYLLIYLTATYPLHPAIA,30.0,AGITPDNNRTQVQNQG,MDRCNNPMARGACYLLIYLTATYPLHPAIAAGITPDNNRTQVQNQG...,1.0,1.233333,0.293333
2194,W3VCV6,SP,0.998977,MDKRSNPLVR,ATSYLLIYLTAIQPL,HPAIA,MDKRSNPLVRATSYLLIYLTAIQPLHPAIA,30.0,AGITPDNDRTQVQHQG,MDKRSNPLVRATSYLLIYLTAIQPLHPAIAAGITPDNDRTQVQHQG...,2.0,1.193333,0.293333
2197,W6HYF7,SP,0.998774,MTN,FIQTVFKIGLFSIFTFFSQL,SSA,MTNFIQTVFKIGLFSIFTFFSQLSSA,26.0,DIDAELIDDMVQVNPL,MTNFIQTVFKIGLFSIFTFFSQLSSADIDAELIDDMVQVNPLISLR...,0.0,1.390000,0.988462


In [ ]:
# Save the cleaned data to an Excel file
toxin_database.to_excel("toxin_data.xlsx")